## Multi Pendulum Cart Problem

In [8]:
%matplotlib inline
import sympy
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from sympy import Matrix, MatrixSymbol, Symbol, symbols, cos, sin, BlockMatrix
from scipy.linalg import solve_continuous_are
from scipy.signal import place_poles, step, lti
from scipy import signal
from tqdm import tqdm
import code.dynamics as dy
from code.utils import plot_respone
from code.controller import Controller 
import code.systems as Z
from scipy.integrate import solve_ivp

## Declare the symbols

In [9]:
ctrl = Controller()
x, theta1, theta2, u = (dy.x, dy.theta1, dy.theta2, dy.u)
x_dot, theta1_dot, theta2_dot = (dy.x_dot, dy.theta1_dot, dy. theta2_dot)
x_ddot, theta1_ddot, theta2_ddot = (dy.x_ddot, dy.theta1_ddot, dy. theta2_ddot)
state = Matrix([x,x_dot, theta1, theta1_dot, theta2, theta2_dot])

# variables
m1, m2, M = (dy.m1, dy.m2, dy.M)
l1, l2 = (dy.l1, dy.l2)
g = dy.g

variables = dy.variables

In [10]:
state

Matrix([
[             x],
[       \dot{x}],
[      \theta_1],
[\dot{\theta_1}],
[      \theta_2],
[\dot{\theta_2}]])

In [11]:
N = dy.N #number of tate
R = Matrix([.003])
Q = sympy.eye(N,N)*150

A_sub = dy.A.subs(variables)
B_sub = dy.B.subs(variables)
A_arr = np.array(A_sub,dtype=np.float32)
B_arr = np.array(B_sub,dtype=np.float32)
Q_arr = np.array(Q,dtype=np.float32)
R_arr = np.array(R,dtype=np.float32)

## LQR 
Solves the continuous algebraic Riccati equation, or CARE, defined as 
$$(A^TP + PA - PBR^{-1}B^TP+Q=0)$$
directly using a Schur decomposition method.

Then compute $K = -R^{-1}B^TP$ the optimal solution to LQR Control problem

In [12]:
K = ctrl.lqr(A_arr,B_arr,Q_arr,R_arr)

### Defining State Space equation
$$
\dot x  = Ax+BKu
$$
$$
Y = Cx
$$
Calculating next state, for given $\delta t$ (dt) time interval.

## Running LQR Controller
* Running for total time = $1000$ sec, and $dt = 0.01$ sec. 
* Initial Condition is defined as 
$$ X(0) = [0,0,30,0,40,0]$$

### Running state space system and plotting without LQR controller

plot_respone(output_arr_wo_control,time_steps,N, plot_title="Response Without Controller")

### Running state space system and plotting with LQR controller


In [13]:
# Running state space with LQR controller
X_states = {x:0,x_dot:0, theta1:30, theta1_dot:0, theta2:40, theta2_dot:0, u:0}
output_arr, time_steps = Z.state_space(A_sub,B_sub,dy.C,K,X_states,state,True)

100%|██████████| 50000/50000 [02:02<00:00, 409.37it/s]


In [ ]:
plot_respone(output_arr,time_steps,N,state,save=False)

### Non Linear system Response

In [ ]:
# Non linear plotting
Y = np.array([0,0,30,0,40,0]).reshape(-1,)
T = np.linspace(0,500,50000)
# ode(T,Y)
ol = solve_ivp(ctrl.ode,(0,500),Y,t_eval=T)
time_steps = np.linspace(0,500,ol.y.shape[1])
plot_respone(ol.y.T,time_steps,N,state,plot_title="Non Linear Controller Response", Plot_filename="Non Linear Controller Response",save=False)

## Printing $P$

In [ ]:

P_round = np.round(P,2)
P_sub = Matrix(P_round)
print(sympy.latex(K))

In [ ]:
print(sympy.latex(P_sub.det()))

In [ ]:
print(P)

In [ ]:
EIG = np.linalg.eigvals(A_arr+B_arr@K_arr)
EIG = np.round(EIG,3)
EIG_sy = Matrix(EIG)